In [1]:
import os
import numpy as np
import pandas as pd

In [23]:
#get reference array of lat and long of all pixels in an area

def get_coords(location):
    
    
    if location == 'antelope':
        
        coords = (-118.4, 34.72, -118.32, 34.79)
        dims = (779, 731)
        
    elif location == 'elsinore':
        
        coords = (-117.44, 33.72, -117.34, 33.8)
        dims = (890, 925)
        
    elif location == 'grassmtn':
        
        coords = (-120.11, 34.75, -120.02, 34.82)
        dims = (779, 822)
    
    
    step_x = abs((coords[0] - coords[2]) / dims[1])
    step_y = abs((coords[1] - coords[3]) / dims[0])
    
    lat, lon = np.mgrid[coords[3]:coords[1]:-step_y, coords[0]:coords[2]:step_x]
    
    
    return lat, lon

## Get latitude and longitude of every pixel predicted to have poppies

In [7]:
#load predictions
preds_2017 = np.load(os.environ['PWD'] + '/shared/probs_2017.npz')
preds_2018 = np.load(os.environ['PWD'] + '/shared/probs_2018.npz')
preds_2019 = np.load(os.environ['PWD'] + '/shared/probs_2019.npz')
preds_2020 = np.load(os.environ['PWD'] + '/shared/probs_2020.npz')

#convert from .npz to dictionaries
preds_2017_dict = {item: preds_2017[item] for item in preds_2017.files}
preds_2018_dict = {item: preds_2018[item] for item in preds_2018.files}
preds_2019_dict = {item: preds_2019[item] for item in preds_2019.files}
preds_2020_dict = {item: preds_2020[item] for item in preds_2020.files}

#merge all years into a single dictionary
preds_dict = {**preds_2017_dict, **preds_2018_dict, **preds_2019_dict, **preds_2020_dict}

In [4]:
#get latitude and longitude of each poppy pixel

lats_dict = {}
longs_dict = {}

for key in preds_dict.keys():
    
    location = key.split('_')[1]
    lat, long = get_coords(location)
    
    arr = preds_dict[key] #binary prediction array
    
    idx = arr > 0.85 #index of poppy pixel predictions model is pretty sure of
    lats_dict[key] = lat[idx]
    longs_dict[key] = long[idx]

In [5]:
#save
np.savez('/home/esther/poppy-finder/data/lats', **lats_dict)
np.savez('/home/esther/poppy-finder/data/longs', **longs_dict)

## Get latitude and longitude of the centroid of each poppy patch

In [2]:
#load centroids
centroids_2017 = np.load(os.environ['PWD'] + '/poppy-finder/data/centroids_2017.npz')
centroids_2018 = np.load(os.environ['PWD'] + '/poppy-finder/data/centroids_2018.npz')
centroids_2019 = np.load(os.environ['PWD'] + '/poppy-finder/data/centroids_2019.npz')
centroids_2020 = np.load(os.environ['PWD'] + '/poppy-finder/data/centroids_2020.npz')

#convert from .npz to dictionaries
centroids_2017_dict = {item: centroids_2017[item] for item in centroids_2017.files}
centroids_2018_dict = {item: centroids_2018[item] for item in centroids_2018.files}
centroids_2019_dict = {item: centroids_2019[item] for item in centroids_2019.files}
centroids_2020_dict = {item: centroids_2020[item] for item in centroids_2020.files}

#merge all years into a single dictionary
centroids_dict = {**centroids_2017_dict, **centroids_2018_dict, **centroids_2019_dict, **centroids_2020_dict}

In [47]:
#get latitude and longitude of each centroid

centroids_lat_long_dict = {}

for key in centroids_dict.keys():
    
    location = key.split('_')[1]
    lat, long = get_coords(location)
    
    #get x and y coordinates of all centroids
    centroids_list = centroids_dict[key]

    #convert to integer (for indexing purposes)
    centroids_list = [[ int(round(cent[0])), int(round(cent[1])) ] for cent in centroids_list]

    #get lat and long coordinates for all centroids in list
    centroids_lat_long_list = [[ lat[cent[0],cent[1]], long[cent[0],cent[1]] ] for cent in centroids_list]
    
    centroids_lat_long_dict[key] = centroids_lat_long_list

In [49]:
#save
np.savez('/home/esther/poppy-finder/data/centroids_lat_long', **centroids_lat_long_dict)